In [1]:
import torch
import torch.optim as optim
from torchinfo import summary

from KSI_models import KSI, ModifiedKSI, LSTM
from KSI_utils import load_KSI_data, train_model, test_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
n_embedding = 100
n_hidden = 100 # 200 in paper, but too intensive for my machine
batch_size = 32
n_epochs = 25
save = True
profile = False
model_type = 'LSTM'

In [3]:
dir = 'data/original/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [4]:
# note_lengths = []
# for data in train_dataloader:
#     n, _, _ = data
#     note_lengths.append(n.shape[1])
# avg_note_size = np.round(np.array(note_lengths).mean()).astype(int)

avg_note_size = 2455

In [5]:
base_model = LSTM(n_words, n_wiki, n_embedding, n_hidden, batch_size)
base_model = base_model.to(DEVICE)
base_summary = summary(base_model, [(batch_size, avg_note_size), 
                                    (batch_size, n_vocab)], 
                       dtypes=[torch.int, torch.float])

base_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─Embedding: 1-1                         [32, 2455, 100]           4,796,200
├─Dropout: 1-2                           [2455, 32, 100]           --
├─LSTM: 1-3                              [2455, 32, 100]           80,800
├─Linear: 1-4                            [32, 344]                 34,744
Total params: 4,911,744
Trainable params: 4,911,744
Non-trainable params: 0
Total mult-adds (G): 6.50
Input size (MB): 1.87
Forward/backward pass size (MB): 125.78
Params size (MB): 19.65
Estimated Total Size (MB): 147.30

In [6]:
optimizer = optim.Adam(base_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_base = train_model(base_model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        scheduler=scheduler,
                        n_epochs=n_epochs,
                        profile=profile, 
                        log_path=f'./log/{model_type}',
                        device=DEVICE,
                        init_hidden=True)

Epoch: 001, Train Recall@10: 0.4407, Val Recall@10: 0.4462, Train Micro F1: 0.0000, Val Micro F1: 0.0000, Train Macro F1: 0.0000, Val Macro F1: 0.0000, Train Micro AUC: 0.9245, Val Micro AUC: 0.9099, Train Macro AUC: 0.4644, Val Macro AUC: 0.4581
Epoch: 002, Train Recall@10: 0.4441, Val Recall@10: 0.4502, Train Micro F1: 0.0000, Val Micro F1: 0.0000, Train Macro F1: 0.0000, Val Macro F1: 0.0000, Train Micro AUC: 0.9279, Val Micro AUC: 0.9137, Train Macro AUC: 0.5853, Val Macro AUC: 0.5927
Epoch: 003, Train Recall@10: 0.5024, Val Recall@10: 0.5075, Train Micro F1: 0.1708, Val Micro F1: 0.1745, Train Macro F1: 0.0067, Val Macro F1: 0.0083, Train Micro AUC: 0.9402, Val Micro AUC: 0.9279, Train Macro AUC: 0.6644, Val Macro AUC: 0.6666
Epoch: 004, Train Recall@10: 0.5562, Val Recall@10: 0.5551, Train Micro F1: 0.2825, Val Micro F1: 0.2833, Train Macro F1: 0.0147, Val Macro F1: 0.0179, Train Micro AUC: 0.9494, Val Micro AUC: 0.9381, Train Macro AUC: 0.7471, Val Macro AUC: 0.7332
Epoch: 005, 

In [7]:
if save:
    torch.save(base_model, f'{dir}{model_type}_model.pt')
if profile:
    print(prof_base.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [8]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_base = test_model(base_model, 
                                                                                                   test_dataloader, 
                                                                                                   wikivec,
                                                                                                   device=DEVICE,
                                                                                                   init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del base_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.6914, Test Micro F1: 0.5432, Test Macro F1: 0.0876, Test Micro AUC: 0.9627, Test Macro AUC: 0.7962


In [9]:
ksi = KSI(n_embedding, n_vocab)
ksi.to(DEVICE)
model = LSTM(n_words, n_wiki, n_embedding, n_hidden, ksi=ksi)
model = model.to(DEVICE)
ksi_summary = summary(model, [(batch_size, avg_note_size), 
                              (batch_size, n_vocab),
                              (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

ksi_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─KSI: 1-1                               --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─Linear: 1-5                            [32, 344]                 34,744
├─KSI: 1-1                               --                        --
│    └─Linear: 2-4                       [32, 344, 100]            1,217,400
│    └─Linear: 2-5                       [32, 344, 100]            10,100
│    └─Linear: 2-6              

In [10]:
optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_ksi = train_model(model, 
                       train_dataloader=train_dataloader,
                       val_dataloader=val_dataloader,
                       wikivec=wikivec,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       n_epochs=n_epochs, 
                       profile=profile, 
                       log_path=f'./log/{model_type}_KSI',
                       device=DEVICE,
                       init_hidden=True)

Epoch: 001, Train Recall@10: 0.6388, Val Recall@10: 0.6383, Train Micro F1: 0.3978, Val Micro F1: 0.3913, Train Macro F1: 0.0476, Val Macro F1: 0.0556, Train Micro AUC: 0.9638, Val Micro AUC: 0.9560, Train Macro AUC: 0.7964, Val Macro AUC: 0.7968
Epoch: 002, Train Recall@10: 0.6828, Val Recall@10: 0.6765, Train Micro F1: 0.4331, Val Micro F1: 0.4237, Train Macro F1: 0.0811, Val Macro F1: 0.0920, Train Micro AUC: 0.9692, Val Micro AUC: 0.9607, Train Macro AUC: 0.8499, Val Macro AUC: 0.8305
Epoch: 003, Train Recall@10: 0.7400, Val Recall@10: 0.7253, Train Micro F1: 0.5338, Val Micro F1: 0.5177, Train Macro F1: 0.1349, Val Macro F1: 0.1430, Train Micro AUC: 0.9762, Val Micro AUC: 0.9673, Train Macro AUC: 0.8899, Val Macro AUC: 0.8586
Epoch: 004, Train Recall@10: 0.7728, Val Recall@10: 0.7531, Train Micro F1: 0.5762, Val Micro F1: 0.5555, Train Macro F1: 0.1803, Val Macro F1: 0.1759, Train Micro AUC: 0.9801, Val Micro AUC: 0.9703, Train Macro AUC: 0.9163, Val Macro AUC: 0.8684
Epoch: 005, 

In [11]:
if save:
    torch.save(model, f'{dir}{model_type}_KSI_model.pt')
if profile:
    print(prof_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [12]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_ksi = test_model(model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7674, Test Micro F1: 0.6162, Test Macro F1: 0.2412, Test Micro AUC: 0.9734, Test Macro AUC: 0.8576


In [13]:
# run modified KSI using frequency vectors rather than binary vectors
dir = 'data/original_freqs/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [14]:
mod_ksi = ModifiedKSI(n_embedding, n_vocab)
mod_ksi.to(DEVICE)
mod_model = LSTM(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi)
mod_model = mod_model.to(DEVICE)
mod_summary = summary(mod_model, [(batch_size, avg_note_size), 
                                  (batch_size, n_vocab),
                                  (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

mod_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─Linear: 1-5                            [32, 344]                 34,744
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                 

In [15]:
optimizer = optim.Adam(mod_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_mod_ksi = train_model(mod_model, 
                           train_dataloader=train_dataloader,
                           val_dataloader=val_dataloader,
                           wikivec=wikivec,
                           optimizer=optimizer,
                           scheduler=scheduler,
                           n_epochs=n_epochs, 
                           profile=profile, 
                           log_path=f'./log/{model_type}_ModifiedKSI',
                           device=DEVICE,
                           init_hidden=True)

Epoch: 001, Train Recall@10: 0.6621, Val Recall@10: 0.6613, Train Micro F1: 0.4390, Val Micro F1: 0.4345, Train Macro F1: 0.0363, Val Macro F1: 0.0429, Train Micro AUC: 0.9653, Val Micro AUC: 0.9585, Train Macro AUC: 0.8057, Val Macro AUC: 0.8209
Epoch: 002, Train Recall@10: 0.7175, Val Recall@10: 0.7153, Train Micro F1: 0.5017, Val Micro F1: 0.4981, Train Macro F1: 0.1002, Val Macro F1: 0.1227, Train Micro AUC: 0.9740, Val Micro AUC: 0.9681, Train Macro AUC: 0.8592, Val Macro AUC: 0.8593
Epoch: 003, Train Recall@10: 0.7456, Val Recall@10: 0.7413, Train Micro F1: 0.5217, Val Micro F1: 0.5162, Train Macro F1: 0.1329, Val Macro F1: 0.1475, Train Micro AUC: 0.9775, Val Micro AUC: 0.9716, Train Macro AUC: 0.8837, Val Macro AUC: 0.8800
Epoch: 004, Train Recall@10: 0.7669, Val Recall@10: 0.7621, Train Micro F1: 0.5607, Val Micro F1: 0.5532, Train Macro F1: 0.1667, Val Macro F1: 0.1714, Train Micro AUC: 0.9803, Val Micro AUC: 0.9743, Train Macro AUC: 0.9062, Val Macro AUC: 0.8925
Epoch: 005, 

In [16]:
if save:
    torch.save(mod_model, f'{dir}{model_type}_ModifiedKSI_model.pt')
if profile:
    print(prof_mod_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [17]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(mod_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del mod_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7625, Test Micro F1: 0.6079, Test Macro F1: 0.2572, Test Micro AUC: 0.9739, Test Macro AUC: 0.8711


In [18]:
# run modified KSI using tfidf vectors rather than binary vectors
dir = 'data/original_tfidf/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [19]:
mod_ksi2 = ModifiedKSI(n_embedding, n_vocab)
mod_ksi2.to(DEVICE)
tfidf_model = LSTM(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi2)
tfidf_model = tfidf_model.to(DEVICE)
tfidf_summary = summary(tfidf_model, [(batch_size, avg_note_size), 
                                      (batch_size, n_vocab),
                                      (n_wiki, n_vocab)], 
                        dtypes=[torch.int, torch.float, torch.float])

tfidf_summary

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [2455, 32, 100]           --
├─LSTM: 1-4                              [2455, 32, 100]           80,800
├─Linear: 1-5                            [32, 344]                 34,744
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                 

In [20]:
optimizer = optim.Adam(tfidf_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_tfidf_ksi = train_model(tfidf_model, 
                             train_dataloader=train_dataloader,
                             val_dataloader=val_dataloader,
                             wikivec=wikivec,
                             optimizer=optimizer,
                             scheduler=scheduler,
                             n_epochs=n_epochs, 
                             profile=profile, 
                             log_path=f'./log/{model_type}_ModifiedKSI_tfidf',
                             device=DEVICE,
                             init_hidden=True)

Epoch: 001, Train Recall@10: 0.6843, Val Recall@10: 0.6828, Train Micro F1: 0.4748, Val Micro F1: 0.4692, Train Macro F1: 0.0531, Val Macro F1: 0.0635, Train Micro AUC: 0.9688, Val Micro AUC: 0.9627, Train Macro AUC: 0.8222, Val Macro AUC: 0.8316
Epoch: 002, Train Recall@10: 0.7240, Val Recall@10: 0.7203, Train Micro F1: 0.5005, Val Micro F1: 0.4938, Train Macro F1: 0.1232, Val Macro F1: 0.1431, Train Micro AUC: 0.9748, Val Micro AUC: 0.9688, Train Macro AUC: 0.8639, Val Macro AUC: 0.8615
Epoch: 003, Train Recall@10: 0.7580, Val Recall@10: 0.7505, Train Micro F1: 0.5613, Val Micro F1: 0.5524, Train Macro F1: 0.1648, Val Macro F1: 0.1681, Train Micro AUC: 0.9784, Val Micro AUC: 0.9725, Train Macro AUC: 0.8879, Val Macro AUC: 0.8818
Epoch: 004, Train Recall@10: 0.7862, Val Recall@10: 0.7800, Train Micro F1: 0.5980, Val Micro F1: 0.5885, Train Macro F1: 0.1821, Val Macro F1: 0.1859, Train Micro AUC: 0.9818, Val Micro AUC: 0.9759, Train Macro AUC: 0.9129, Val Macro AUC: 0.8982
Epoch: 005, 

In [21]:
if save:
    torch.save(tfidf_model, f'{dir}{model_type}_ModifiedKSI_tfidf_model.pt')
if profile:
    print(prof_tfidf_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [22]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(tfidf_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE,
                                                                                                  init_hidden=True)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del tfidf_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7545, Test Micro F1: 0.6045, Test Macro F1: 0.2591, Test Micro AUC: 0.9698, Test Macro AUC: 0.8503
